In [ ]:
import logging
import os
import nest_asyncio

nest_asyncio.apply()


from dotenv import load_dotenv
from openai import AsyncAzureOpenAI

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

load_dotenv(verbose=True)

api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key= os.getenv("AZURE_OPENAI_API_KEY")
deployment_name = 'gpt-4o'
api_version = '2024-02-15-preview' # this might change in the future

aAzureOpenclient =  AsyncAzureOpenAI(
        api_key=api_key,  
        api_version=api_version,
        base_url=f"{api_base}/openai/deployments/{deployment_name}"
    )

In [ ]:
systemPrompt = { "role": 
                        "system", 
                "content": '''
                            You are a helpful assistant and you are a good video player. 
                            You know teh video games very well. You can give professional answer about video game's questions or screenshots. 
                            But you have answer the question or screenshot or enrich the content based on the given context and images.
                            The context format is a list which contain a few  like this:
                            
                            context1：
                                "caption": "The caption of the image",
                                "content": "The content of the image",
                                "ocrContent": "The ocr content of the image"
                                "images": "https://url1"

                            -----
                            context2：
                                "caption": "The caption of the image",
                                "content": "The content of the image",
                                "ocrContent": "The ocr content of the image"
                                "images": "https://url2"

                            -----
                            context3：
                                "caption": "The caption of the image",
                                "content": "The content of the image",
                                "ocrContent": "The ocr content of the image"
                                "images": "https://url3"
                                
                            You can use the caption,content and ocrContent to answer the question if they are related to the question.
                            And if you choose them to answer the question, you should return the related images url.
                            The response should be json format, have two parts: answer and related images url.
                            like this:
                            {
                                "answer": "The answer of the question",
                                "images": ["https://url1", "https://url2"]
                            }
                        ''' 
                }


In [37]:
import sys

# 假设 multiModelsEmbedding.py 文件在项目的根目录
sys.path.append(os.path.abspath("/Users/huqianghui/Downloads/git_temp/multi-model-data-process/"))


from search_utils import get_search_results_by_image_and_text,get_search_results_by_text,get_search_results_by_image

##### query text only

In [ ]:
queryText = "DNF手游伤害为什么是黄字？"

In [39]:
results = await get_search_results_by_text(queryText)

conext = ""
for i, result in enumerate(results, start=1):
    conext += f"context{i}：\n\n"
    conext += f"caption:\n{result['caption']}\n\n"
    conext += f"content:\n{result['content']}\n\n"
    conext += f"ocrContent:\n{result['ocrContent']}\n\n"
    conext += f"imageUrl:\n{result['imageUrl']}\n\n"
    conext += "-------\n\n"

userPrompt = ''' user question is : {queryText} 

                answer the question based the context: {context}
'''.format(queryText=queryText,context=conext)

response = await aAzureOpenclient.chat.completions.create(
        model=deployment_name,
        response_format={ "type": "json_object" },
        messages=[
            systemPrompt,
            { "role": "user", "content": [  
                { 
                    "type": "text", 
                    "text": userPrompt
                }
            ] } 
        ],
        max_tokens=800 
    )

result = response.choices[0].message.content

from IPython.display import display, Image, Markdown
import json

resultDict = json.loads(result)
answer = resultDict['answer']
image_urls = resultDict['images']

# 显示答案文本
display(Markdown(f"**Answer:**\n\n{answer}"))

# 显示图像
for url in image_urls:
    display(Image(url=url))


2024-08-28 12:56:16,383 - INFO - HTTP Request: POST https://openai-hu-non-product-test.openai.azure.com//openai/deployments/text-embedding-ada-002/embeddings?api-version=2024-02-01 "HTTP/1.1 200 OK"
2024-08-28 12:56:16,384 - INFO - Getting text embedding for DNF手游伤害为什么是黄字？
2024-08-28 12:56:17,725 - INFO - Request URL: 'https://ai-search-hu-west-us-3.search.windows.net/indexes('multi-model-index-zh-cn')/docs/search.post.search?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '48866'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': 'dc4a2906-64f9-11ef-b95b-aee50117d95a'
    'User-Agent': 'azsdk-python-search-documents/11.6.0b4 Python/3.11.9 (macOS-14.6.1-arm64-arm-64bit)'
A body is sent with the request
2024-08-28 12:56:19,594 - INFO - Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.met

**Answer:**

在DNF手游中，'黄字'伤害代表的是你的主攻击力增加后的伤害值。根据context1的信息，'n%伤害 攻击增加伤害' 这种词条会让你的攻击力增加n%，并显示在主伤害里面。例如，如果你的攻击力是100，增加5%的伤害，你的主伤害就会变成105。这种类型的伤害会显示为黄字。

#### query text & image

In [ ]:
queryText = "DNF手游伤害为什么是黄字？"
picture_url = "https://img2.tapimg.com/moment/etag/FvhNYMQT78nnCjAvBqHvY40FcH46.jpeg"

#### 从 AI Search 获取到相关内容，组装成prompt里面的context

In [ ]:

results = await get_search_results_by_image_and_text(picture_url,queryText)

conext = ""
for i, result in enumerate(results, start=1):
    conext += f"context{i}：\n\n"
    conext += f"caption:\n{result['caption']}\n\n"
    conext += f"content:\n{result['content']}\n\n"
    conext += f"ocrContent:\n{result['ocrContent']}\n\n"
    conext += f"imageUrl:\n{result['imageUrl']}\n\n"
    conext += "-------\n\n"

In [ ]:
userPrompt = ''' user question is : {queryText} 
                and with the image.

                answer the question based context is : {context}
'''.format(queryText=queryText,context=conext)

In [36]:
response = await aAzureOpenclient.chat.completions.create(
        model=deployment_name,
        response_format={ "type": "json_object" },
        messages=[
            systemPrompt,
            { "role": "user", "content": [  
                { 
                    "type": "text", 
                    "text": userPrompt
                },
                { 
                    "type": "image_url",
                    "image_url": {
                        "url": picture_url
                    }
                }
            ] } 
        ],
        max_tokens=800 
    )

result = response.choices[0].message.content
print(result)

2024-08-28 12:52:56,471 - INFO - HTTP Request: POST https://openai-hu-non-product-test.openai.azure.com//openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview "HTTP/1.1 200 OK"


{
    "answer": "在DNF手游中，增加伤害的词条（n%伤害增加）显示为黄色，因为这种增加伤害会直接加到你的主伤害中。例如，如果你的攻击力是100，加上5%的伤害增加，那么你的攻击力就会变成105，这就是所谓的黄字显示。相对于增加伤害的词条，还有一种“攻击时附加n属性伤害”的词条，这类词条显示为白字，因为其附加的属性伤害会在主伤害条下显示，不直接加到主伤害中。",
    "images": ["https://img2.tapimg.com/moment/etag/FvhNYMQT78nnCjAvBqHvY40FcH46.jpeg"]
}


In [40]:
from IPython.display import display, Image, Markdown
import json

resultDict = json.loads(result)
answer = resultDict['answer']
image_urls = resultDict['images']

# 显示答案文本
display(Markdown(f"**Answer:**\n\n{answer}"))

# 显示图像
for url in image_urls:
    display(Image(url=url))

**Answer:**

在DNF手游中，'黄字'伤害代表的是你的主攻击力增加后的伤害值。根据context1的信息，'n%伤害 攻击增加伤害' 这种词条会让你的攻击力增加n%，并显示在主伤害里面。例如，如果你的攻击力是100，增加5%的伤害，你的主伤害就会变成105。这种类型的伤害会显示为黄字。